In [191]:
import numpy as np
import h5py
import tensorflow as tf
#from tensorflow.python.ops import rnn, rnn_cell

In [192]:
def one_hot(i):
    a = [0,0,0,0]
    a[i-769] = 1;
    return a

In [193]:
# Load subject data and store the reshaped data in X_sub[i], y_sub[i] 
X_sub = []
y_sub = []
for i in np.arange(9):
    data = h5py.File('./project_datasets/project_datasets/A0'+str(i+1)+'T_slice.mat', 'r')
    X_sub.append(np.copy(data['image']))
    y_sub.append(np.copy(data['type']))
    X_sub[i] = X_sub[i][:, :22, :]
    X_sub[i] = X_sub[i].transpose([0,2,1])
    y_sub[i] = y_sub[i][0,0:X_sub[i].shape[0]:1]
    y_sub[i] = np.asarray(y_sub[i], dtype=np.int32)
    y_sub[i] = [one_hot(j) for j in y_sub[i]]
    y_sub[i] = np.asarray(y_sub[i], dtype = np.int8)
    print(X_sub[i].shape)


(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)
(288, 1000, 22)


In [194]:
# Remove NaN
for i in np.arange(9):
    y_sub[i] = y_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]
    X_sub[i] = X_sub[i][~np.isnan(X_sub[i]).any(axis=(1,2))]

In [195]:
# Data Normalization
def normalize(dataset):
    mu = np.mean(dataset,axis=0)
    sigma = np.std(dataset,axis=0)
    return (dataset - mu)/ sigma

In [196]:
# For training across all subjects
# Shuffle the data, sample 20% for testing, and the rest for training
X = []
y = []

X = np.concatenate([X_sub[i] for i in np.arange(9)])
y = np.concatenate([y_sub[i] for i in np.arange(9)])
num_trial = X.shape[0]
index_shuffle = np.arange(num_trial)
np.random.shuffle(index_shuffle)
X_shuffle = X[index_shuffle]
y_shuffle = y[index_shuffle]

training_index = int(np.floor(num_trial * (1 - 0.2)))
#validation_index = training_index + int(np.floor(num_trial * 0.2))
# test_index = num_trial

X_train = X_shuffle[0:training_index]
y_train = y_shuffle[0:training_index]
X_train_new_shape = X_train.reshape([X_train.shape[0], 20, 50, 22]).reshape([X_train.shape[0], 20, -1])
# X_val = X_shuffle[training_index:validation_index]
# y_val = y_shuffle[training_index:validation_index]
# y_val_steps = np.tile(y_val,((X_train.shape[1]),1))
X_test = X_shuffle[training_index:num_trial]
X_test_new_shape = X_test.reshape([X_test.shape[0], 20, 50, 22]).reshape([X_test.shape[0], 20, -1])
y_test = y_shuffle[training_index:num_trial]
y_test_steps = np.tile(y_test,((X_train.shape[1]),1))

X_train = normalize(X_train)
X_train_new_shape = normalize(X_train_new_shape)
#X_val = normalize(X_val)
X_test = normalize(X_test)

print(X_train.shape)
print(X_train_new_shape.shape)
print(y_train.shape)
print(X_test.shape)
print(X_test_new_shape.shape)
print(y_test.shape)
print(y_test_steps.shape)

(2046, 1000, 22)
(2046, 20, 1100)
(2046, 4)
(512, 1000, 22)
(512, 20, 1100)
(512, 4)
(512000, 4)


In [197]:
# For training within each subject[i]
# Shuffle the data, sample 50 trials for testing, and the rest for training -->
X_train_sub = []
X_train_new_shape_sub = []

y_train_sub = []
y_train_new_shape_sub = []

X_test_sub = []
X_test_new_shape_sub = []

y_test_sub = []
y_test_new_shape_sub = []

y_test_steps_sub = []

for i in np.arange(9):
    num_trial_sub = X_sub[i].shape[0]
    training_index_sub = num_trial_sub - 50
    
    index_shuffle_sub = np.arange(num_trial_sub)
    np.random.shuffle(index_shuffle_sub)
    
    X_shuffle_sub = X_sub[i][index_shuffle_sub]
    y_shuffle_sub = y_sub[i][index_shuffle_sub]
    X_train_sub.append(X_shuffle_sub[0:training_index_sub])
    y_train_sub.append(y_shuffle_sub[0:training_index_sub])
    X_test_sub.append(X_shuffle[training_index_sub:num_trial_sub])
    y_test_sub.append(y_shuffle[training_index_sub:num_trial_sub])
    
    # new shape
    
    X_shuffle_new_shape_sub = X_sub[i][index_shuffle_sub].reshape([-1, 20, 50, 22]).reshape([-1, 20, 1100])
    X_train_new_shape_sub.append(X_shuffle_new_shape_sub[0:training_index_sub])
    X_test_new_shape_sub.append(X_shuffle_new_shape_sub[training_index_sub:num_trial_sub])
    '''
    y_shuffle_new_shape_sub = y_sub[i][index_shuffle_sub].reshape([-1, 20, 50, 22]).reshape([-1, 20, 1100])
    y_train_new_shape_sub.append(y_shuffle_new_shape_sub[0:training_index_sub])
    y_test_new_shape_sub.append(y_shuffle_new_shape_sub[training_index_sub:num_trial_sub])
    '''
    
    y_test_steps_sub.append(np.tile(y_test_sub[i],((X_train_sub[i].shape[1]),1)))

    X_train_sub[i] = normalize(X_train_sub[i])
    #X_train_new_shape_sub[i] = normalize(X_train_new_shape_sub[i])
    X_test_sub[i] = normalize(X_test_sub[i])
    X_test_new_shape_sub[i] = normalize(X_test_new_shape_sub[i])
    
    print(X_train_sub[i].shape)
    print(X_train_new_shape_sub[i].shape)
    print(X_test_sub[i].shape)
    print(X_test_new_shape_sub[i].shape)
    
    print(y_test_steps_sub[i].shape)
    print()

(237, 1000, 22)
(237, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(236, 1000, 22)
(236, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(236, 1000, 22)
(236, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(234, 1000, 22)
(234, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(232, 1000, 22)
(232, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(235, 1000, 22)
(235, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(238, 1000, 22)
(238, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(232, 1000, 22)
(232, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)

(228, 1000, 22)
(228, 20, 1100)
(50, 1000, 22)
(50, 20, 1100)
(50000, 4)



In [198]:
# For training across all subjects: Hyperparameters setting
tf.reset_default_graph()

learning_rate = 0.0001
training_epochs = 30
batch_size = 35
total_batches = (X_train.shape[0]//batch_size)
print(total_batches)

58


In [199]:
# parameters about CNN model from paper
# idea: input of CovLayer is a 2d matrix with size 1000 * 22
image_height = 1000
image_width = 22

In [200]:
# placeholders
x_holder = tf.placeholder(tf.float32, [None, image_height, image_width, 1])
y_holder = tf.placeholder(tf.float32, [None, 4])

In [201]:
# layer 1: 25, 10 * 1 filter + 25, 25 * 22 filter + maxpool 3*1
layer1_params = {
    'filter_set_1':[10, 1, 1, 25],
    'filter_set_2':[1, 22, 25, 25]
}

W1_1 = tf.Variable(tf.random_normal(layer1_params['filter_set_1'], stddev=0.1))
b1_1 = tf.Variable(tf.random_normal([25], stddev=0.1))
W1_2 = tf.Variable(tf.random_normal(layer1_params['filter_set_2'], stddev=0.1))
b1_2 = tf.Variable(tf.random_normal([25], stddev=0.1))

In [202]:
# Define computation graph - layer 1
a1_1 = tf.nn.conv2d(x_holder, W1_1, strides=[1,1,1,1], padding='VALID') + b1_1
print(a1_1.shape)
a1_2 = tf.nn.conv2d(a1_1, W1_2, strides=[1,1,1,1], padding='VALID') + b1_2
print(a1_2.shape)
z1 = tf.nn.max_pool(a1_2, ksize=[1,3,1,1], strides=[1,3,1,1], padding='VALID')
print(z1.shape)

(?, 991, 22, 25)
(?, 991, 1, 25)
(?, 330, 1, 25)


In [203]:
# Define computation graph - layer 2
layer2_params = {
    'filter':[10,1,25,50],
    'max_pool':[1,2,1,1]
}
W2 = tf.Variable(tf.random_normal(layer2_params['filter'],stddev=0.1))
b2 = tf.Variable(tf.random_normal([50]))
a2 = tf.nn.conv2d(z1, W2, strides=[1,1,1,1], padding='VALID') + b2
print(a2.shape)
h2 = tf.nn.relu(a2)
print(h2.shape)
z2 = tf.nn.max_pool(h2, ksize=layer2_params['max_pool'], strides=[1,2,1,1], padding='VALID')
print(z2.shape)

(?, 321, 1, 50)
(?, 321, 1, 50)
(?, 160, 1, 50)


In [204]:
# Define computation graph - layer 3
layer3_params = {
    'filter':[10, 1, 50, 100],
    'max_pool':[1,2,1,1]
}
W3 = tf.Variable(tf.random_normal(layer3_params['filter'],stddev=0.1))
b3 = tf.Variable(tf.random_normal([100]))
a3 = tf.nn.conv2d(z2, W3, strides=[1,1,1,1], padding='VALID') + b3
print(a3.shape)
h3 = tf.nn.relu(a3)
print(h3.shape)
z3 = tf.nn.max_pool(h3, ksize=layer3_params['max_pool'], strides=[1,2,1,1], padding='VALID')
print(z3.shape)

(?, 151, 1, 100)
(?, 151, 1, 100)
(?, 75, 1, 100)


In [205]:
# Define computation graph - layer 4
layer4_params = {
    'filter':[10, 1, 100, 200],
    'max_pool':[1,2,1,1]
}

W4 = tf.Variable(tf.random_normal(layer4_params['filter'],stddev=0.1))
b4 = tf.Variable(tf.random_normal([200]))
a4 = tf.nn.conv2d(z3, W4, strides=[1,1,1,1], padding='VALID') + b4
print(a4.shape)
h4 = tf.nn.relu(a4)
print(h4.shape)
z4 = tf.nn.max_pool(h4, ksize=layer4_params['max_pool'], strides=[1,2,1,1], padding='VALID')
print(z4.shape)

(?, 66, 1, 200)
(?, 66, 1, 200)
(?, 33, 1, 200)


In [206]:
# Define computation graph - fc layer
num_features = z4.get_shape()[1:4].num_elements()
flaten_layer = tf.reshape(z4, [-1, num_features])

W5 = tf.Variable(tf.random_normal(shape = [num_features, 4], stddev=0.1))
b5 = tf.Variable(tf.random_normal(shape = [4], stddev=0.1))
h5 = tf.matmul(flaten_layer, W5) + b5
z_out = tf.nn.relu(h5)
print(z_out.shape)

(?, 4)


In [207]:
# compute loss
y_pred = tf.nn.softmax(z_out)
y_pred_class = tf.argmax(y_pred, axis=1)
cross_entropy = tf.losses.softmax_cross_entropy(y_holder, logits=z_out)
regularizer = tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4)
cost = tf.reduce_mean(cross_entropy + 0.05 * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_pred = tf.equal(tf.argmax(y_pred, axis = 1),tf.argmax(y_holder,axis = 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))

In [208]:
# train
display_step = 25
'''
sample_x = X_train[15:50, :]
batch_shape = sample_x.shape
print(sample_x.shape)
batch_x = sample_x.reshape([batch_shape[0], batch_shape[1], batch_shape[2], 1])
print(batch_x.shape)

'''
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):
            offset = (b * batch_size) % (y_train.shape[0] - batch_size)
            sample_x = X_train[offset:(offset + batch_size), :]
            batch_shape = sample_x.shape
            batch_x = sample_x.reshape([batch_shape[0], batch_shape[1], batch_shape[2], 1])
            batch_y = y_train[offset:(offset + batch_size), :]
            
            _, acc, loss = sess.run([optimizer, accuracy, cost], feed_dict={x_holder: batch_x, y_holder: batch_y})
            if (b % display_step == 0):
                print('training accuracy = %.2f, training loss = %f' % (acc, loss))
        test_shape = X_test.shape
        x_feed = X_test.reshape([test_shape[0], test_shape[1], test_shape[2], 1])
        test_accuracy = sess.run(accuracy, feed_dict={x_holder:x_feed, y_holder:y_test})
        print('Epoch %d: test accuracy = %f' % (epoch, test_accuracy))

training accuracy = 0.31, training loss = 73.997986
training accuracy = 0.14, training loss = 65.938446
training accuracy = 0.31, training loss = 64.804619
Epoch 0: test accuracy = 0.236328
training accuracy = 0.23, training loss = 64.426697
training accuracy = 0.14, training loss = 63.213554
training accuracy = 0.31, training loss = 61.971409
Epoch 1: test accuracy = 0.236328
training accuracy = 0.23, training loss = 61.570477
training accuracy = 0.14, training loss = 60.312523
training accuracy = 0.31, training loss = 59.052643
Epoch 2: test accuracy = 0.236328
training accuracy = 0.23, training loss = 58.650120
training accuracy = 0.14, training loss = 57.396446
training accuracy = 0.31, training loss = 56.151993
Epoch 3: test accuracy = 0.236328
training accuracy = 0.23, training loss = 55.756180
training accuracy = 0.14, training loss = 54.528118
training accuracy = 0.31, training loss = 53.314800


KeyboardInterrupt: 